## FDW Crop Production Data Profiling - Zimbabwe

In [2]:
import os, sys, glob, json
from itertools import product, compress, chain
from functools import reduce
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import requests
import numpy as np
import pandas as pd
import geopandas as gpd
from tools import save_hdf, save_npz, load_npz, PrintAdminUnits, PlotAdminShapes
from tools import FDW_PD_Sweeper, FDW_PD_AvalTable, FDW_PD_Compiling, FDW_PD_GrainTypeAgg, FDW_PD_ValidateFnidName
from tools import FDW_PD_CreateAdminLink, FDW_PD_RatioAdminLink, FDW_PD_ConnectAdminLink
from tools import FDW_PD_CaliSeasonYear
from tools_graphic import PlotBarProduction, PlotLinePAY, PlotHeatCropSystem, PlotHeatSeasonData
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
pd.options.mode.chained_assignment = None

In [3]:
# CPCV2 grain code ------------------------------ #
grain_code = pd.read_hdf('../data/crop/grain_cpcv2_code.hdf')
product_category = grain_code[['product', 'product_category']].set_index('product').to_dict()['product_category']
# ----------------------------------------------- #

# Load FEWS NET administrative boundaries ------- #
epsg = 'epsg:32736' # Zimbabwe
fn_shapes = sorted(glob.glob('../data/shapefile/fewsnet/ZW_Admin?_????.shp')+
                   glob.glob('../data/shapefile/fewsnet/ZW_Crop_Regions_????.shp'))
shape_all = []
for fn in fn_shapes:
    name = fn.split('/')[-1].split('.')[0]
    exec('%s = gpd.read_file("%s").to_crs("%s")' % (name, fn, epsg))
    exec('%s["area"] = %s["geometry"].area/10**6' % (name, name))
    exec('shape_all.append(%s)' % (name))
shape_all = pd.concat(shape_all, axis=0).reset_index(drop=True)
PrintAdminUnits(shape_all)
# ----------------------------------------------- #

# FDW API host address -------------------------- #
host = 'https://fdw.fews.net'
#auth = tuple(json.loads(open('token.json', "r").read()))
parameters = {
    'format': 'json',
    'country': 'Zimbabwe',
    'product': 'R011',
    'survey_type': 'crop:best'
}
endpoint = '/api/cropproductionindicatorvalue/'
response = requests.get(host + endpoint, params=parameters, proxies={})#auth=auth,
response.raise_for_status()
df = pd.DataFrame.from_records(response.json())
df_origin = df.copy(deep=True)
# ----------------------------------------------- #

- FEWS NET admin shapefiles ------------------- #
| year	 | Admin1   | # units   | Admin2   | # units   | Admin3   | # units   |
| 1981	 | ZW1981A1 | 8	| ZW1981A2	| 58	| nan	| 0	|
| 2007	 | ZW2007A1 | 10	| ZW2007A2	| 84	| nan	| 0	|
| 2008	 | ZW2008A1 | 10	| ZW2008A2	| 84	| nan	| 0	|
| 2009	 | ZW2009A1 | 10	| ZW2009A2	| 85	| nan	| 0	|
| 2011	 | ZW2011A1 | 10	| ZW2011A2	| 89	| nan	| 0	|
| 2018	 | ZW2018A1 | 10	| ZW2018A2	| 89	| nan	| 0	|
----------------------------------------------- #


In [4]:
# Manual Pre-processing before Sweeping --------- #
# 1. Default setting 
# a) None-type crop production system
df.loc[df['crop_production_system'].isna(), 'crop_production_system'] = 'none'
df.loc[df['crop_production_system'] == '', 'crop_production_system'] = 'none'
# b) None-type population group
df.loc[df['population_group'].isna(), 'population_group'] = 'none'
df.loc[df['population_group'] == '', 'population_group'] = 'none'
# ----------------------------------------------- #

# FDW Production Data Inspection ---------------- #
df, df_raw = FDW_PD_Sweeper(df, area_priority='Area Planted')
table_dict = FDW_PD_AvalTable(df, shape_all)
# ----------------------------------------------- #

# FEWS NET Shapefile comparison ----------------- #
shape_used = pd.concat([ZW_Admin2_2011], axis=0)
PlotAdminShapes(shape_used, label=True)
# ----------------------------------------------- #

- Remove missing records ---------------------- #
Orignial data points: 56,056
Removed 22,673 "Missing Value" points
0/14,014 "Area Harvested" points are retained.
11,359/14,014 "Area Planted" points are retained.
11,286/14,014 "Quantity Produced" points are retained.
10,738/14,014 "Yield" points are retained.
Current data points: 33,383

- Minor changes are applied ------------------- #

- Basic information --------------------------- #
Data period: 1981 - 2022
7 grain types are found: Maize Grain (White), Millet, Millet (Bulrush), Millet (Finger), Rice (Paddy), Sorghum, Sorghum (Brown)
1 seasons are found: Main (04-01)
2 crop production system are found: Communal (PS), All (PS)
Data sources include:
[1] Ministry of Agriculture, AGRITEX, Zimbabwe --- Agritex field survey reports, Zimbabwe
[2] MoA, Zimbabwe --- Ministry of Agriculture Crop Assessment Report 2nd round, Zimbabwe
[3] MoA, Zimbabwe --- Ministry of Agriculture Crop Assessment Report, Zimbabwe
Administrative-1 fnids: 0
Admin

IndexError: index 0 is out of bounds for axis 0 with size 0

- Zimbabwe crop seasonal calendar

![FEWS NET](../figures/crop_calendar/seasonal-calendar-zimbabwe.png)
<img src="../figures/crop_calendar/Zimbabwe_-_Crop_calendar.jpg"  width="60%">

- FDW data consists of `ZW2011A2`.

| Year  | Admin-1  | # units  | Admin-2  | # units |
| :---: | :----:   | :----:   | :----:   | :---:   |
| 1981	 | ZW1981A1 | 8	| ZW1981A2	| 58	|
| 2007	 | ZW2007A1 | 10	| ZW2007A2	| 84	|
| 2008	 | ZW2008A1 | 10	| ZW2008A2	| 84	|
| 2009	 | ZW2009A1 | 10	| ZW2009A2	| 85	|
| 2011	 | ZW2011A1 | 10	| **`ZW2011A2`**	| 89	|

- Comparison between admin boundaries.

![image](https://github.com/chc-ucsb/gscd/blob/main/figures/ZW_admin_shapes.png?raw=true)

- **FDW data consists of only 3 years of records**.
- **`ZW2011A2`** is used to represent the current admin-level 2 crop data.
- Zimbabwe has a single crop season: `Main`.
- Zimbabwe has no population group(s).
- Zimbabwe has no crop production system(s).

In [ ]:
# Define the latest shapefile ------------------- #
latest_level = 2
shape_latest = ZW_Admin2_2011.copy().to_crs('epsg:4326')
# ----------------------------------------------- #

# # Validation of FNIDs and Names ----------------- #
# df = FDW_PD_ValidateFnidName(df, shape_used, shape_latest)
# # ----------------------------------------------- #

# # FDW Production Data Compiling ----------------- #
# area, prod = FDW_PD_Compiling(df, shape_used)
# area_all, prod_all = area.copy(), prod.copy()
# mdx_pss = area.columns.droplevel([0,1]).unique()
# # ----------------------------------------------- #